In [5]:
search_for = 122
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.00395512580871582
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 436374755
type: [1, 1, 1, 1, 122] 122
cases of this type: 221533456
100000 17.35103497773552
200000 31.90332050019947
300000 33.973442569526725
400000 34.12444781587249
500000 33.87206507892822
600000 32.590052612373825
700000 31.76662160011538
800000 32.20311673533532
900000 26.75695426658209
1000000 18.249605428641726
1100000 32.16403479666464
1200000 31.685983564916008
1300000 31.833402542789678
1400000 33.17847333374977
1500000 31.751773591188098
1600000 31.63885816328082
1700000 31.798463790123225
1800000 29.566107489501647
1900000 14.441230024333791
2000000 31.199807212429047
2100000 30.343285229388982
2200000 30.52975733411953
2300000 32.1353961184627
2400000 30.88567243768447
2500000 30.637626272758013
2600000 29.97449765819398
2700000 29.638346403538435
2800000 13.272261832058524
2900000 29.81758439311911
3000000 30.2258

29500000 31.22608681888916
29600000 29.015975343657406
29700000 29.123355105463624
29800000 27.322870065381263
29900000 26.859394175783823
30000000 12.352387961861305
30100000 27.194516586648863
30200000 26.33755802089252
30300000 27.968302289850048
30400000 29.214117675278285
30500000 30.763899111424326
30600000 29.986136140756518
30700000 15.834101032713276
30800000 29.76907020326175
30900000 30.835729454087137
31000000 31.797507167640344
31100000 30.49043282023472
31200000 16.0322234605671
31300000 31.815995500583906
31400000 30.755430614175083
31500000 30.728812623811788
31600000 29.22595474052211
31700000 27.976247004369526
31800000 15.610468118572786
31900000 23.710435456396816
32000000 27.731934840319596
32100000 29.24466428719364
32200000 29.922388245695267
32300000 31.599349059518758
32400000 34.761191248270784
32500000 15.84791297889044
32600000 31.58444697699192
32700000 32.23001954898655
32800000 30.31888444382843
32900000 32.97177013499511
33000000 14.570394703014824
33100

59200000 17.592389308193923
59300000 14.399235731915025
59400000 11.239321178239475
59500000 11.73232575746472
59600000 16.385703744219008
59700000 11.978511362711128
59800000 14.623649264850899
59900000 23.190182064850717
60000000 25.187030379470006
60100000 18.55936450806492
60200000 21.517382841994227
60300000 17.212743313710195
60400000 11.294457886732747
60500000 7.008620610061148
60600000 16.654001376659043
60700000 20.136963030570048
60800000 12.281754971501888
60900000 4.9898709899616565
61000000 16.6636175931313
61100000 12.472593088210703
61200000 11.658320249796352
61300000 9.960521378865549
61400000 19.445154250302593
61500000 14.900988920701053
61600000 13.5905645261845
61700000 13.65103850984327
61800000 13.141744641444344
61900000 13.004367902241302
62000000 12.810123311449468
62100000 12.968369390109963
62200000 11.743593837214988
62300000 8.82403271284908
62400000 12.557426786908493
62500000 12.359674566226387
62600000 11.052386837900409
62700000 4.950001861471855
6280

88900000 26.033778834701337
89000000 26.364734759469748
89100000 27.562326316356327
89200000 25.74985064947592
89300000 24.257920191048026
89400000 24.201862242836906
89500000 24.61018133522579
89600000 24.436750996572137
89700000 19.895348185610512
89800000 15.577775885356695
89900000 14.589808034557462
90000000 19.341931348571407
90100000 13.447937289338123
90200000 23.663595915047736
90300000 23.494461956008784
90400000 23.826056463126584
90500000 25.541963230607546
90600000 24.60369213774367
90700000 27.680286168183756
90800000 27.36884849971238
90900000 24.952307186812217
91000000 26.077745156152588
91100000 22.838301407291844
91200000 24.134780068224902
91300000 22.820617534910514
91400000 23.170400510484487
91500000 24.239393475948383
91600000 11.459720694726792
91700000 17.373331362882762
91800000 14.630849200903548
91900000 13.238311987512317
92000000 22.596983715492787
92100000 23.37814328841565
92200000 23.179137239645968
92300000 23.222415821693986
92400000 23.7193638629867

117900000 30.597192439574044
118000000 31.809088593703557
118100000 28.910652756078186
118200000 25.11266476615216
118300000 25.376584023040266
118400000 24.090896903069424
118500000 23.880124622046036
118600000 24.371117510119042
118700000 23.698179858373045
118800000 22.873500835256312
118900000 15.175856724216253
119000000 3.8764313454020325
119100000 16.094845688327133
119200000 23.424386794056907
119300000 24.04800471930054
119400000 24.63473982307764
119500000 24.055895444935796
119600000 24.09951481115413
119700000 24.531284271078597
119800000 25.350953491468122
119900000 25.425606965841705
120000000 22.20506066171966
120100000 21.149067266838887
120200000 19.961893569349794
120300000 20.181353902158403
120400000 20.03627792229774
120500000 19.009674477504998
120600000 19.276216126949095
120700000 14.54990360796002
120800000 3.8031665743069154
120900000 10.97903491862548
121000000 20.090142325937975
121100000 20.233479570022105
121200000 20.74241591184169
121300000 20.9439142918

...

319700000 5.893879495779337
319800000 3.461972417681055
319900000 4.407704811415829
320000000 5.783017750609448
320100000 5.7425227762729625
320200000 6.177083221319289
320300000 5.7524044690839355
320400000 5.755143530436162
320500000 2.6455825248907177
320600000 2.7408156628983713
320700000 5.804536154504822
320800000 5.977736740114404
320900000 6.921340584147116
321000000 6.155313581296219
321100000 6.278576319488511
321200000 6.639460385518968
321300000 6.078894415245466
321400000 6.608247975324805
321500000 6.470919069456786
321600000 2.9899724490343464
321700000 4.868679971527401
321800000 5.724953697166353
321900000 5.725976981509421
322000000 5.785591275335044
322100000 5.685175142989606
322200000 5.370859065026081
322300000 3.431168641199774
322400000 1.8182281391629778
322500000 5.806359128821896
322600000 6.0455299983835955
322700000 6.214040409179587
322800000 6.1815497254011955
322900000 6.567050852206428
323000000 6.588413500290156
323100000 5.284695575476852
323200000 6.

348000000 3.48167042670488
348100000 3.641211317735321
348200000 2.727291294393361
348300000 2.9203797788063786
348400000 3.671665909580118
348500000 2.79324568918401
348600000 3.821808596559849
348700000 3.4747720396272372
348800000 3.9032145707792045
348900000 2.905211226978848
349000000 3.4173160364141624
349100000 2.3658105104157627
349200000 2.9571108685490657
349300000 3.5525950382675866
349400000 2.4993183040935496
349500000 3.1146772515848737
349600000 3.519751165173474
349700000 3.761387384318749
349800000 2.8906657175636026
349900000 3.3208527710670825
350000000 2.5256268599132
350100000 1.939858602718019
350200000 3.6194224359692004
350300000 2.1380918926472665
350400000 3.362585372861895
350500000 4.087555796288921
350600000 3.5107348991532326
350700000 2.6380382642877667
350800000 3.129185476241019
350900000 3.682145746962518
351000000 2.125209553290053
351100000 3.332330775995768
351200000 2.633890677065581
351300000 2.7850238082475127
351400000 4.311848193882591
35150000

376600000 1.528971976691428
376700000 1.7431167005275885
376800000 1.446018425077362
376900000 1.6079776319555243
377000000 1.4297016120069623
377100000 0.8908772669082615
377200000 1.734644109012647
377300000 0.8556750312258303
377400000 1.4158072867529758
377500000 1.5942257469622594
377600000 1.38267137903137
377700000 1.5121241305308044
377800000 2.0849532628498473
377900000 2.790901158040375
378000000 1.9057444137000374
378100000 1.364657951364653
378200000 0.7739327317005694
378300000 1.3203070735345817
378400000 1.4997529661544062
378500000 1.9385027878928185
378600000 1.4782642395292553
378700000 1.524040811652713
378800000 1.6994132768180072
378900000 0.9420095854554839
379000000 1.327190801826606
379100000 0.7728890333474577
379200000 1.3291349548617037
379300000 1.0042513155351613
379400000 1.365521868370533
379500000 1.6596894357260963
379600000 1.4361597626257177
379700000 1.3750728088143467
379800000 1.1410921932877105
379900000 1.2727389193109604
380000000 0.786046946873

403900000 0.4498734349984593
404000000 0.4484142689014971
404100000 0.6733995093429253
404200000 0.6840446650277078
404300000 0.569379825093329
404400000 0.45076958832880193
404500000 0.5307716213185125
404600000 0.6832227209898332
404700000 0.6573535382905967
404800000 0.4865750071355204
404900000 0.39728622187045215
405000000 0.6273668165904449
405100000 0.6879430013394421
405200000 0.61471717600137
405300000 0.3071872846007281
405400000 0.5519963912984696
405500000 0.6448879812522231
405600000 0.6080626092138125
405700000 0.48564431520203083
405800000 0.32759272090885044
405900000 0.5808379654559758
406000000 0.674188068748484
406100000 0.6290901534469228
406200000 0.3159995759833786
406300000 0.5622323319504361
406400000 0.6261385905732015
406500000 0.7279823128091262
406600000 0.6458745326144861
406700000 0.38176349158447986
406800000 0.7110250280469027
406900000 0.4635497287167046
407000000 0.11601539423659443
407100000 0.11269031219102608
407200000 0.11701545573746164
407300000 

430200000 0.06961707093690833
430300000 0.09419828097473913
430400000 0.06232702356985212
430500000 0.11716202603224583
430600000 0.06885665983703401
430700000 0.08239392162057757
430800000 0.07955701309195161
430900000 0.06560702344282138
431000000 0.08663093442848326
431100000 0.06157942600549592
431200000 0.08195917401227354
431300000 0.05441715965206425
431400000 0.075641386214902
431500000 0.05778543080097105
431600000 0.06698018173143268
431700000 0.06709276994140942
431800000 0.05487689016903771
431900000 0.0701980899758041
432000000 0.05015504983960589
432100000 0.07238758908225762
432200000 0.050688227092921735
432300000 0.06130596183939444
432400000 0.05083062675392628
432500000 0.05064118436198433
432600000 0.05542774350144466
432700000 0.04351451570350925
432800000 0.05368965577054024
432900000 0.04449468978626529
433000000 0.04902189325231976
433100000 0.03727400199335813
433200000 0.04440035335474875
433300000 0.042265050743367935
433400000 0.036942802471806604
433500000 